In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import traceback
import os.path
import csv
from csv import writer
import urllib.request

In [2]:
def initializeDriver():
    """
    Initializes the web browser driver.

    Returns
    -------
    driver
    """
    
    chrome_path = "chromedriver"

    driver=webdriver.Chrome(chrome_path)
    
    return driver

def cleanText(text):
    """
    Replaces non-alphanumeric characters in text with a space
    """
    import re
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    text = text.strip()
    
    return text

def scrape_article_text(driver):
                    
    try:
        #text = driver.find_element_by_xpath('/html/body/div[1]/div/div[4]/div[1]/article/div[1]').text

        params =  driver.find_elements_by_xpath('/html/body/div[1]/div/div[4]/div[1]/article/p')

        for p in params:
            print(p.text)

    except Exception as ex:
        try:
            # text = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[1]/article/div/div/div/div").text
            params =  driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/div[1]/article/p')

            for p in params:
                print(p.text)

        except Exception as ex:
            print(str(ex))

def scrape_timestamp(driver):
    
    timestamp = None
    
    try:
        timestamp = driver.find_element_by_xpath("/html/head/meta[@name='article:published_time']").get_attribute('content')

    except Exception as ex:
        try:
            timestamp = driver.find_element_by_xpath("/html/head/meta[@property='og:article:published_time']").get_attribute('content')

        except Exception as ex:
            print(str(ex))
    
    return timestamp

def scrape_category(driver):
    
    category = None
    
    try:
        category = driver.find_element_by_xpath("/html/head/meta[@name='analyticsAttributes.topicChannel']").get_attribute('content')

    except Exception as ex:
        print(str(ex))
    
    return category

def scrape_subcategory(driver):
    
    subcategory = None
    
    try:
        subcategory = driver.find_element_by_xpath("/html/head/meta[@name='analyticsAttributes.topicSubChannel']").get_attribute('content')

    except Exception as ex:
        print(str(ex))
    
    return subcategory

def scrape_page(link, driver):
    
    driver.get(link)
    
    article_list = driver.find_elements_by_tag_name('article')
    
    hrefs = []
    
    data_list = []
    
    for article in article_list:
        try:
            title = cleanText(article.find_element_by_xpath('./div[2]/a/h3').text)
            href = article.find_element_by_xpath('./div[2]/a').get_attribute('href')
            desc = cleanText(article.find_element_by_xpath('./div[2]/p').text)
            date = article.find_element_by_xpath('./div[2]/time/span').text
            hrefs.append(href)
            
            data_list.append([href,title,desc,date])
            
        except Exception as ex:
            print(str(ex))
            
    df = pd.DataFrame(data_list)
    df.columns = ['href','title','desc','date']
    df['timestamp'] = pd.Series(index=df.index, dtype='object')
    df['category'] = pd.Series(index=df.index, dtype='object')
    df['sub_category'] = pd.Series(index=df.index, dtype='object')
    
    i = 0
    for href in hrefs:
        i+=1
        print(str(i))
        driver.get(href)
        #time.sleep(1)
        
        idx = df.index[df['href'] == href].tolist()[0]
        
        timestamp = scrape_timestamp(driver)
        df['timestamp'].iloc[idx] = timestamp
        
        category = scrape_category(driver)
        df['category'].iloc[idx] = category
        
        sub_category = scrape_subcategory(driver)
        df['sub_category'].iloc[idx] = sub_category
        
    return df

In [16]:
driver = initializeDriver()

for i in range(769,771):
    print("Page: "+str(i))
    link = 'https://www.reuters.com/news/archive/japan?view=page&page='+str(i)+'&pageSize=10'
    df = scrape_page(link, driver)
    print(df.head(2))
    df.to_csv("scraped_news.csv",mode='a', header=False, index=False)

Page: 769
Message: no such element: Unable to locate element: {"method":"xpath","selector":"./div[2]/time/span"}
  (Session info: chrome=90.0.4430.212)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"./div[2]/p"}
  (Session info: chrome=90.0.4430.212)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"./div[2]/p"}
  (Session info: chrome=90.0.4430.212)

1
2
3
4
5
6
7
8
9
10
                                                href  \
0  https://www.reuters.com/article/us-tech-ces-au...   
1  https://www.reuters.com/article/us-japan-boj-i...   

                                               title  \
0  Automakers not Silicon Valley lead in driverle...   
1  Japan central bank turns activist investor to ...   

                                                desc         date  \
0  Automakers not technology companies are in the...  JAN 05 2016   
1  Japan s central bank which dominates the domes...  JAN 03 2016   

          